<a href="https://colab.research.google.com/github/akankshakusf/Project-DeepLearning-English-to-French-Translation/blob/master/Neural_Machine_Translation_with_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import ML packages
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.metrics import confusion_matrix,roc_curve
import pathlib
import io
import re
import string
import time

#import DL package
import cv2
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer,Dense, Flatten, InputLayer, BatchNormalization, Bidirectional, Dropout, Input, Embedding, TextVectorization
from tensorflow.keras.layers import SimpleRNN, Conv1D, LSTM, GRU
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy, TopKCategoricalAccuracy, TopKCategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from tensorboard.plugins import projector

# Data Preparation

## Data Download

In [2]:
!wget https://www.manythings.org/anki/fra-eng.zip

--2025-04-13 10:26:58--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7943074 (7.6M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.57M  8.38MB/s    in 0.9s    

2025-04-13 10:26:59 (8.38 MB/s) - ‘fra-eng.zip’ saved [7943074/7943074]



In [3]:
!unzip "/content/fra-eng.zip" -d "/content/dataset/"

Archive:  /content/fra-eng.zip
  inflating: /content/dataset/_about.txt  
  inflating: /content/dataset/fra.txt  


## Kaggle Dataset

In [6]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d dhruvildave/en-fr-translation-dataset

Dataset URL: https://www.kaggle.com/datasets/dhruvildave/en-fr-translation-dataset
License(s): ODbL-1.0
en-fr-translation-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!unzip "/content/en-fr-translation-dataset.zip" -d "/content/dataset/"

Archive:  /content/en-fr-translation-dataset.zip
  inflating: /content/dataset/en-fr.csv  


In [8]:
dataset = tf.data.experimental.CsvDataset(
  "/content/dataset/en-fr.csv",
  [
    tf.string,
    tf.string
  ],
)

## Data Processing

In [10]:
text_dataset=tf.data.TextLineDataset("/content/dataset/fra.txt")

In [11]:
#review dataset
for i in text_dataset.take(3):
  print(i)

tf.Tensor(b'Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)', shape=(), dtype=string)
tf.Tensor(b'Go.\tMarche.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)', shape=(), dtype=string)
tf.Tensor(b'Go.\tEn route !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)', shape=(), dtype=string)


In [12]:
# #lets skip a max number of records and check what max length we find
# for i in text_dataset.skip(190000):
#   print(len(tf.strings.split(i," ")))

Streaming output truncated to the last 5000 lines.
38
35
30
34
30
30
32
35
35
31
36
30
30
31
34
35
33
29
35
29
29
31
27
31
30
31
35
32
33
33
28
33
33
31
32
31
30
37
32
33
31
31
31
34
34
32
33
34
32
33
33
36
31
32
28
32
29
31
28
31
30
30
32
29
29
31
30
36
35
36
38
28
32
31
32
34
35
33
34
35
34
33
36
33
31
26
30
34
32
32
37
33
33
33
32
27
29
31
31
34
34
30
30
32
31
33
28
34
31
29
31
28
30
29
31
29
33
35
36
37
35
34
30
28
27
30
33
30
34
30
34
32
30
30
29
28
33
31
29
29
33
28
28
31
33
32
31
35
29
27
33
30
34
32
32
31
31
32
30
35
31
32
30
36
36
34
33
33
34
33
29
35
31
34
31
33
31
33
28
33
36
30
38
33
30
32
28
28
33
32
30
35
36
32
34
31
31
36
32
34
34
35
32
36
35
33
35
35
31
31
31
31
34
34
34
35
36
35
36
37
31
33
35
32
34
33
32
32
31
32
34
34
34
35
35
33
33
35
31
30
29
34
34
34
35
35
35
35
35
35
35
32
32
36
29
30
29
33
30
39
41
33
29
28
30
30
30
29
34
28
28
38
28
30
29
35
33
38
29
32
33
33
32
34
30
31
30
30
30
36
36
34
28
32
30
26
32
31
30
33
33
30
39
33
29
34
30
31
36
36
36
36
29
34
32
33
3

- Since i saw that the max len of the sentence is 107. I am going to go ahead with a sequence length of 64 as we also have french letters

In [15]:
VOCAB_SIZE= 20000
SEQUENCE_LENGTH=64
EMBEDDING_DIM = 300

- Create vectorizer layer to create vectors
- reference :https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization

In [23]:
english_vectorize_layer = TextVectorization (
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length = SEQUENCE_LENGTH
)

In [21]:
french_vectorize_layer = TextVectorization (
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length = SEQUENCE_LENGTH
)

- Look at this sample data from dataset and get rid of tabs--->  \t
'Go.\tVa !\tCC-BY 2.0 (France)

In [27]:
def selector (input_text):
  split_text=tf.strings.split(input_text, '\t')
  #after splitting collect english and french separately
  return split_text[0:1], split_text[1:2]

In [28]:
#map text_dataset to selector above function
split_dataset = text_dataset.map(selector)

In [29]:
# review the data
for i in split_dataset.take(2):
  print(i)

(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Va !'], dtype=object)>)
(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Go.'], dtype=object)>, <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'Marche.'], dtype=object)>)


- Notice how nicely english and french text have been separated now

In [31]:
# now lets attach this Vectorizer to split_dataset to get the vocabulary list
english_training_dataset = split_dataset.map(lambda x,y:x) ##input is x, y and output is x
english_vectorize_layer.adapt(english_training_dataset) ##adapth the vectorizer layer to training data

KeyboardInterrupt: 

In [ ]:
# now lets attach this Vectorizer to split_dataset to get the vocabulary list
french_training_dataset = split_dataset.map(lambda x,y:x) ##input is x, y and output is x
english_vectorize_layer.adapt(french_training_dataset) ##adapth the vectorizer layer to training data

# Modeling

# Evaluation

# Testing